In [1]:
from platform import python_version

print(python_version())

3.10.8


In [2]:
import open3d as o3d
import meshio
import plotly.graph_objects as go
import numpy as np

mesh_monki = o3d.io.read_triangle_mesh('untitled.obj')
trimodel_mesh = meshio.read('untitled.obj')
vertices_trimodel = trimodel_mesh.points
triangles_trimodel = trimodel_mesh.cells_dict['triangle']

x=vertices_trimodel[:, 0]
y=vertices_trimodel[:, 1]
z=vertices_trimodel[:, 2]
i=triangles_trimodel[:, 0]
j=triangles_trimodel[:, 1]
k=triangles_trimodel[:, 2]

edges = []
for face in triangles_trimodel:
    for ind in range(len(face)):
        if ind < len(face)-1:
            edges.append(
                (face[ind], face[ind+1])
            )
        else:
            edges.append(
                (face[ind], face[0])
            )
            
xe=[]
ye=[]
ze=[]

for edge in edges:
    xe.extend([x[edge[0]], x[edge[1]], None])
    ye.extend([y[edge[0]], y[edge[1]], None])
    ze.extend([z[edge[0]], z[edge[1]], None])

trace_monki_mesh = go.Mesh3d(x=x, y=y, z=z, color='white', alphahull=3, #flatshading=True,
                               i=i,
                               j=j,
                               k=k,)

trace_monki_edges = go.Scatter3d(x=xe, y=ye, z=ze, mode='lines', line=dict(color='pink', width=1))

vs = 0.25
vg_monki = o3d.geometry.VoxelGrid.create_from_triangle_mesh(input=mesh_monki, voxel_size=vs)
monki_vox = vg_monki.get_voxels()

monki_vox_idx = [_.grid_index for _ in monki_vox]
monki_vox_ptx = np.asarray([vg_monki.get_voxel_bounding_points(_) for _ in monki_vox_idx])
edge_x = []
edge_y = []
edge_z = []

for box in monki_vox_ptx:
    for pair in [(0,1), (0,2), (0,4), (1,3), (1,5), (2,6), (3,7), (4,5), (4,6), (5,7), (6, 7), (2,3)]:
        edge_x.extend([box[pair[0]][0], box[pair[1]][0], None])
        edge_y.extend([box[pair[0]][1], box[pair[1]][1], None])
        edge_z.extend([box[pair[0]][2], box[pair[1]][2], None])

trace_edges = go.Scatter3d(x=edge_x, y=edge_y, z=edge_z, mode='lines')

data = [trace_monki_mesh, trace_monki_edges, trace_edges]

layout = go.Layout(width=1000,
                   height=1000,
                   showlegend=False,
                   hovermode='closest')


fig = go.Figure(data=data, layout=layout)

fig.update_scenes(camera_projection_type='orthographic')

fig.show()

ModuleNotFoundError: No module named 'open3d'